# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv


# Dataset Loading

In [27]:
train = pd.read_csv(r'D:\Project Phase II\Dataset\finaltrain.csv',encoding='cp1252')
test = pd.read_csv(r'D:\Project Phase II\Dataset\finaltest.csv',encoding='cp1252') #test_set1 
autrain = pd.read_csv(r'D:\Project Phase II\Dataset\UNSW_NB15_training-set.csv',encoding='cp1252') #train_set2
autest = pd.read_csv(r'D:\Project Phase II\Dataset\UNSW_NB15_testing-set.csv',encoding='cp1252') #test_set2

C:\Users\sathi\AppData\Local\Temp\ipykernel_7480\1785680868.py:1: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(r'D:\Project Phase II\Dataset\finaltrain.csv',encoding='cp1252')
C:\Users\sathi\AppData\Local\Temp\ipykernel_7480\1785680868.py:2: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(r'D:\Project Phase II\Dataset\finaltest.csv',encoding='cp1252') #test_set1


In [28]:
completeDf = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')

columns_to_match = ['srcip', 'sport', 'dstip', 'dsport', 'Label']

#Change the values of Stime and Ltime in test and replace with its corresponding values in completeDf based on match with columns_to_match
for i in range(len(test)):
    for j in range(len(completeDf)):
        if (test['srcip'][i] == completeDf['srcip'][j]) and (test['sport'][i] == completeDf['sport'][j]) and (test['dstip'][i] == completeDf['dstip'][j]) and (test['dsport'][i] == completeDf['dsport'][j]) and (test['Label'][i] == completeDf['Label'][j]):
            test['Stime'][i] = completeDf['Stime'][j]
            test['Ltime'][i] = completeDf['Ltime'][j]
            break

for i in range(len(train)):
    for j in range(len(completeDf)):
        if (train['srcip'][i] == completeDf['srcip'][j]) and (train['sport'][i] == completeDf['sport'][j]) and (train['dstip'][i] == completeDf['dstip'][j]) and (train['dsport'][i] == completeDf['dsport'][j]) and (train['Label'][i] == completeDf['Label'][j]):
            train['Stime'][i] = completeDf['Stime'][j]
            train['Ltime'][i] = completeDf['Ltime'][j]
            break

#Save as csv
train.to_csv(r'D:\Project Phase II\Dataset\finaltrain.csv', index=False)
test.to_csv(r'D:\Project Phase II\Dataset\finaltest.csv', index=False)

C:\Users\sathi\AppData\Local\Temp\ipykernel_7480\490001905.py:1: DtypeWarning: Columns (1,3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  completeDf = pd.read_csv(r'D:\Project Phase II\Dataset\completedataset.csv',encoding='cp1252')
C:\Users\sathi\AppData\Local\Temp\ipykernel_7480\490001905.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Stime'][i] = completeDf['Stime'][j]
C:\Users\sathi\AppData\Local\Temp\ipykernel_7480\490001905.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Ltime'][i] = completeDf['Ltime'][j]
C:\Users\sathi\AppData\Local\Temp\ipykernel_7480\

KeyboardInterrupt: 

In [32]:
import pandas as pd

# Define columns to match
columns_to_match = ['srcip', 'sport', 'dstip', 'dsport', 'Label']

# Merge test and completeDf DataFrames based on columns_to_match
merged_df = pd.merge(test, completeDf, on=columns_to_match, suffixes=('_test', '_complete'), how='left')

# Update Stime and Ltime in test with corresponding values from completeDf

test['Stime'] = merged_df['Stime_complete']
test['Ltime'] = merged_df['Ltime_complete']
# Drop the extra columns from the merged DataFrame
merged_df.drop(columns=['Stime_complete', 'Ltime_complete'], inplace=True)

In [35]:
test.sport

0        48987
1        44230
2        58214
3        53815
4        42546
         ...  
34711    48878
34712     9979
34713    54643
34714    32281
34715    38931
Name: sport, Length: 34716, dtype: int64

In [34]:
merged_df.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto_test', 'state_test',
       'dur_test', 'sbytes_test', 'dbytes_test', 'sttl_test', 'dttl_test',
       'sloss_test', 'dloss_test', 'service_test', 'Sload_test', 'Dload_test',
       'Spkts_test', 'Dpkts_test', 'swin_test', 'dwin_test', 'stcpb_test',
       'dtcpb_test', 'smeansz_test', 'dmeansz_test', 'trans_depth_test',
       'res_bdy_len_test', 'Sjit_test', 'Djit_test', 'Stime_test',
       'Ltime_test', 'Sintpkt_test', 'Dintpkt_test', 'tcprtt_test',
       'synack_test', 'ackdat_test', 'ct_state_ttl_test',
       'ct_flw_http_mthd_test', 'is_ftp_login_test', 'ct_ftp_cmd_test',
       'ct_srv_src_test', 'ct_srv_dst_test', 'ct_dst_ltm_test',
       'ct_src_ ltm_test', 'ct_src_dport_ltm_test', 'ct_dst_sport_ltm_test',
       'ct_dst_src_ltm_test', 'attack_cat_test', 'Label', 'proto_complete',
       'state_complete', 'dur_complete', 'sbytes_complete', 'dbytes_complete',
       'sttl_complete', 'dttl_complete', 'sloss_complete', 'dloss

In [33]:
test['Stime']

0        1.421968e+09
1                 NaN
2                 NaN
3        1.421947e+09
4                 NaN
             ...     
34711             NaN
34712             NaN
34713             NaN
34714    1.421971e+09
34715    1.421945e+09
Name: Stime, Length: 34716, dtype: float64

# Data Preprocessing

## Preprocess user-made train-test datasets

In [6]:
todrop = ['sloss', 'dloss','stcpb', 'dtcpb', 'trans_depth', 'Stime', 'Ltime','tcprtt', 'ct_flw_http_mthd', 
        'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat']
        
todropButIncStimeLtime = ['sloss', 'dloss','stcpb', 'dtcpb', 'trans_depth','tcprtt', 'ct_flw_http_mthd', 
        'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat']

reducedTrain = train.drop(todrop, axis = 1)
reducedTest = test.drop(todropButIncStimeLtime, axis = 1)
reducedTrain = reducedTrain.drop_duplicates()
reducedTest = reducedTest.drop_duplicates()

trainAttributes = reducedTrain.drop(['srcip','sport','dstip','dsport','Label'], axis = 1)
trainLabel = reducedTrain['Label']
testAttributes = reducedTest.drop(['srcip','sport','dstip','dsport','Label'], axis = 1)
testLabel = reducedTest['Label']

train = reducedTrain
test = reducedTest

train = train.drop_duplicates(['srcip','sport', 'dstip', 'dsport', 'sttl', 'dttl', 'swin', 'dwin'], keep = 'last')
test = test.drop_duplicates(['srcip','sport', 'dstip', 'dsport', 'sttl', 'dttl', 'swin', 'dwin'], keep = 'last')
testWithStimeLtime = test[['srcip', 'sport', 'dstip', 'dsport', 'Stime', 'Ltime']]
test = test.drop(['Stime','Ltime'], axis = 1)

In [4]:
print("Train Set:\n",train.Label.value_counts())
print("Test Set:\n", test.Label.value_counts())

Train Set:
 Label
0    39990
1    39866
Name: count, dtype: int64
Test Set:
 Label
1    17229
0    17201
Name: count, dtype: int64


## Preprocess Author's Train-Test dataset

Use stcpb, dtcpb to substitute the unavailablity of ip port

In [7]:
autraindrop = ['ï»¿id', 'rate', 'sloss', 'dloss','trans_depth','tcprtt','ct_srv_src','ct_dst_ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
        'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
        'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']


autrain = autrain.drop(autraindrop, axis = 1)
autrain.rename(columns={'dinpkt':'Dintpkt','djit':'Djit','dload':'Dload','dpkts':'Dpkts','label':'Label','sinpkt':'Sintpkt','sjit':'Sjit', 'sload':'Sload', 'spkts':'Spkts','dmean':'dmeansz','response_body_len':'res_bdy_len', 'smean':'smeansz'}, inplace=True)
autrain = autrain[(autrain['proto']=='tcp') | (autrain['proto']=='udp') | (autrain['proto']=='ospf')]
autrain = autrain[(autrain['service']=='ssh') | (autrain['service']=='ftp-data ')  | (autrain['service']=='ftp') | (autrain['service']=='-') | (autrain['service']=='dns') | (autrain['service']=='smtp') | (autrain['service']=='http') | (autrain['service']=='radius') | (autrain['service']=='pop3') ]
autrain = autrain[(autrain['state'] == 'CON') | (autrain['state'] == 'RST') | (autrain['state'] == 'FIN') | (autrain['state'] == 'ACC') | (autrain['state'] == 'REQ') | (autrain['state'] == 'INT')]

autrain['proto'].replace("tcp", 0, inplace = True)
autrain['proto'].replace("udp", 1, inplace = True)
autrain['proto'].replace("ospf", 2, inplace = True)

autrain['service'].replace("ssh", 0, inplace = True)
autrain['service'].replace("ftp-data", 1, inplace = True)
autrain['service'].replace("ftp", 2, inplace = True)
autrain['service'].replace("-", 3, inplace = True)
autrain['service'].replace("dns", 4, inplace = True)
autrain['service'].replace("smtp", 5, inplace = True)
autrain['service'].replace("http", 6, inplace = True)
autrain['service'].replace("radius", 7, inplace = True)
autrain['service'].replace("pop3", 8, inplace = True)

autrain['state'].replace("CON", 0, inplace = True)
autrain['state'].replace("RST", 1, inplace = True)
autrain['state'].replace("FIN", 2, inplace = True)
autrain['state'].replace("ACC", 3, inplace = True)
autrain['state'].replace("REQ", 4, inplace = True)
autrain['state'].replace("INT", 5, inplace = True)

for column in autrain.columns:
        if column != 'Label':
                col_mean = sum(autrain[column]) / len(autrain[column])
                col_std = (sum((x - col_mean) ** 2 for x in autrain[column]) / len(autrain[column])) ** 0.5
                autrain[column] = [(x - col_mean) / col_std for x in autrain[column]]


autrain = autrain.drop_duplicates(['sttl', 'dttl', 'swin', 'dwin','stcpb','dtcpb'], keep = 'last')
autrainAttributes = autrain.drop(['Label'], axis = 1)
autrainAttributes = autrainAttributes[trainAttributes.columns]
autrainLabel = autrain['Label']

In [89]:
autrain.columns

Index(['dur', 'proto', 'service', 'state', 'Spkts', 'Dpkts', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'Sload', 'Dload', 'Sintpkt', 'Dintpkt',
       'Sjit', 'Djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'synack', 'ackdat',
       'smeansz', 'dmeansz', 'res_bdy_len', 'ct_state_ttl', 'Label'],
      dtype='object')

In [90]:
print("Train Set:\n",autrain.Label.value_counts())


Train Set:
 Label
1    36961
0    34967
Name: count, dtype: int64


In [8]:
autestdrop = ['ï»¿id', 'rate', 'sloss', 'dloss','trans_depth','tcprtt','ct_srv_src','ct_dst_ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
        'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
        'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']

autest = autest.drop(autestdrop, axis = 1)
#Change column names of autest
autest.rename(columns={'dinpkt':'Dintpkt','djit':'Djit','dload':'Dload','dpkts':'Dpkts','label':'Label','sinpkt':'Sintpkt','sjit':'Sjit', 'sload':'Sload', 'spkts':'Spkts','dmean':'dmeansz','response_body_len':'res_bdy_len', 'smean':'smeansz'}, inplace=True)
autest = autest[(autest['proto']=='tcp') | (autest['proto']=='udp') | (autest['proto']=='ospf')]
autest = autest[(autest['service']=='ssh') | (autest['service']=='ftp-data ')  | (autest['service']=='ftp') | (autest['service']=='-') | (autest['service']=='dns') | (autest['service']=='smtp') | (autest['service']=='http') | (autest['service']=='radius') | (autest['service']=='pop3') ]
autest = autest[(autest['state'] == 'CON') | (autest['state'] == 'RST') | (autest['state'] == 'FIN') | (autest['state'] == 'ACC') | (autest['state'] == 'REQ') | (autest['state'] == 'INT')]

autest['proto'].replace("tcp", 0, inplace = True)
autest['proto'].replace("udp", 1, inplace = True)
autest['proto'].replace("ospf", 2, inplace = True)

autest['service'].replace("ssh", 0, inplace = True)
autest['service'].replace("ftp-data", 1, inplace = True)
autest['service'].replace("ftp", 2, inplace = True)
autest['service'].replace("-", 3, inplace = True)
autest['service'].replace("dns", 4, inplace = True)
autest['service'].replace("smtp", 5, inplace = True)
autest['service'].replace("http", 6, inplace = True)
autest['service'].replace("radius", 7, inplace = True)
autest['service'].replace("pop3", 8, inplace = True)

autest['state'].replace("CON", 0, inplace = True)
autest['state'].replace("RST", 1, inplace = True)
autest['state'].replace("FIN", 2, inplace = True)
autest['state'].replace("ACC", 3, inplace = True)
autest['state'].replace("REQ", 4, inplace = True)
autest['state'].replace("INT", 5, inplace = True)

for column in autest.columns:
        if column != 'Label':
                col_mean = sum(autest[column]) / len(autest[column])
                col_std = (sum((x - col_mean) ** 2 for x in autest[column]) / len(autest[column])) ** 0.5
                autest[column] = [(x - col_mean) / col_std for x in autest[column]]


autest = autest.drop_duplicates(['sttl', 'dttl', 'swin', 'dwin','stcpb','dtcpb'], keep = 'last')
autestAttributes = autest.drop(['Label'], axis = 1)
autestAttributes = autestAttributes[trainAttributes.columns]
autestLabel = autest['Label']

In [62]:
print("Author's Testing Set:\n", autest.Label.value_counts())

Author's Testing Set:
 Label
0    24354
1    13671
Name: count, dtype: int64


# Input Transformation

## Training

In [9]:
nodes = set()
trainUnique = train
for i in range(len(train)):
    src = str(train['srcip'].iloc[i])+':'+ str(train['sport'].iloc[i]) + ':' + str(train['sttl'].iloc[i])+':'+str(train['swin'].iloc[i])
    dst = str(train['dstip'].iloc[i])+':'+str(train['dsport'].iloc[i]) + ':' + str(train['dttl'].iloc[i])+':'+str(train['dwin'].iloc[i])
    nodes.add(src)
    nodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
train_tuples = set()
for i in range(len(train)):
    train_tuples.add((train['srcip'].iloc[i], train['sport'].iloc[i], train['dstip'].iloc[i], train['dsport'].iloc[i], train['sttl'].iloc[i], train['swin'].iloc[i], train['dttl'].iloc[i], train['dwin'].iloc[i]))

In [10]:
xLookUp = {}
x = []
count = 0
edge_index = []
edge_attr = []
for i in range(len(train)):
    src = str(train['srcip'].iloc[i])+':'+ str(train['sport'].iloc[i]) + ':' + str(train['sttl'].iloc[i])+':'+str(train['swin'].iloc[i])
    dst = str(train['dstip'].iloc[i])+':'+str(train['dsport'].iloc[i]) + ':' + str(train['dttl'].iloc[i])+':'+str(train['dwin'].iloc[i])
    if src not in xLookUp:
        xLookUp[src] = count
        # x.append([int(train['sport'].iloc[i]), int(train['sttl'].iloc[i]), int(train['swin'].iloc[i])])
        x.append([int(train['sttl'].iloc[i]), int(train['swin'].iloc[i])])
        count += 1
    if dst not in xLookUp:
        xLookUp[dst] = count
        # x.append([int(train['dsport'].iloc[i]), int(train['dttl'].iloc[i]), int(train['dwin'].iloc[i])])
        x.append([int(train['dttl'].iloc[i]), int(train['dwin'].iloc[i])])
        count += 1
    edge_index.append([xLookUp[src], xLookUp[dst]])
    edge_attr.append(list(train.iloc[i].drop(['srcip', 'sport', 'dstip', 'dsport', 'sttl', 'dttl','swin','dwin','Label']).values))


# Convert x to tensor
x_tensor = torch.tensor(x, dtype=torch.float)

# Convert edge_index to tensor
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)

print(x_tensor.shape, edge_index_tensor.shape, edge_attr_tensor.shape)

torch.Size([97138, 2]) torch.Size([2, 79856]) torch.Size([79856, 20])


## Testing

In [11]:
testnodes = set()
testUnique = test
for i in range(len(test)):
    src = str(test['srcip'].iloc[i])+':'+ str(test['sport'].iloc[i]) + ':' + str(test['sttl'].iloc[i])+':'+str(test['swin'].iloc[i])
    dst = str(test['dstip'].iloc[i])+':'+str(test['dsport'].iloc[i]) + ':' + str(test['dttl'].iloc[i])+':'+str(test['dwin'].iloc[i])
    testnodes.add(src)
    testnodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
test_tuples = set()
for i in range(len(test)):
    test_tuples.add((test['srcip'].iloc[i], test['sport'].iloc[i], test['dstip'].iloc[i], test['dsport'].iloc[i], test['sttl'].iloc[i], test['swin'].iloc[i], test['dttl'].iloc[i], test['dwin'].iloc[i]))

In [12]:
xtestLookUp = {}
xtest = []
count = 0
edge_index_test = []
edge_attr_test = []
for i in range(len(test)):
    src = str(test['srcip'].iloc[i])+':'+ str(test['sport'].iloc[i]) + ':' + str(test['sttl'].iloc[i])+':'+str(test['swin'].iloc[i])
    dst = str(test['dstip'].iloc[i])+':'+str(test['dsport'].iloc[i]) + ':' + str(test['dttl'].iloc[i])+':'+str(test['dwin'].iloc[i])
    if src not in xtestLookUp:
        xtestLookUp[src] = count
        # xtest.append([int(test['sport'].iloc[i]), int(test['sttl'].iloc[i]), int(test['swin'].iloc[i])])
        xtest.append([int(test['sttl'].iloc[i]), int(test['swin'].iloc[i])])
        count += 1
    if dst not in xtestLookUp:
        xtestLookUp[dst] = count
        # xtest.append([int(test['dsport'].iloc[i]), int(test['dttl'].iloc[i]), int(test['dwin'].iloc[i])])
        xtest.append([int(test['dttl'].iloc[i]), int(test['dwin'].iloc[i])])
        count += 1
    edge_index_test.append([xtestLookUp[src], xtestLookUp[dst]])
    edge_attr_test.append(list(test.iloc[i].drop(['srcip', 'sport', 'dstip', 'dsport', 'sttl', 'dttl','swin','dwin','Label']).values))


# Convert x to tensor
x_test_tensor = torch.tensor(xtest, dtype=torch.float)

# Convert edge_index to tensor
edge_index_test_tensor = torch.tensor(edge_index_test, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_test_tensor = torch.tensor(edge_attr_test, dtype=torch.float)

print(x_test_tensor.shape, edge_index_test_tensor.shape, edge_attr_test_tensor.shape)

torch.Size([42906, 2]) torch.Size([2, 34430]) torch.Size([34430, 20])


## Author's Training Dataset

Since srcip, sport, dstip, dsport information is not available, we proceed with having each node feature value as 0, and assume each edge as an unique connection between two imaginary nodes, we add stcpb and dtcpb as placeholders 

In [13]:
autrainnodes = set()
autrainUnique = autrain
for i in range(len(autrain)):
    src = str(autrain['sttl'].iloc[i])+':'+str(autrain['swin'].iloc[i]) + str(autrain['stcpb'].iloc[i])
    dst = str(autrain['dttl'].iloc[i])+':'+str(autrain['dwin'].iloc[i]) + str(autrain['dtcpb'].iloc[i])
    autrainnodes.add(src)
    autrainnodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
autrain_tuples = set()
for i in range(len(autrain)):
    autrain_tuples.add((autrain['sttl'].iloc[i], autrain['swin'].iloc[i], autrain['dttl'].iloc[i], autrain['dwin'].iloc[i], autrain['stcpb'].iloc[i], autrain['dtcpb'].iloc[i]))

In [14]:
xautrainLookUp = {}
xautrain = []
count = 0
edge_index_autrain = []
edge_attr_autrain = []

for i in range(len(autrain)):
    src = str(autrain['sttl'].iloc[i])+':'+str(autrain['swin'].iloc[i]) + str(autrain['stcpb'].iloc[i])
    dst = str(autrain['dttl'].iloc[i])+':'+str(autrain['dwin'].iloc[i]) + str(autrain['dtcpb'].iloc[i])
    if src not in xautrainLookUp:
        xautrainLookUp[src] = count
        # xautrain.append([0,int(autrain['sttl'].iloc[i]), int(autrain['swin'].iloc[i])])
        xautrain.append([int(autrain['sttl'].iloc[i]), int(autrain['swin'].iloc[i])])
        count += 1
    if dst not in xautrainLookUp:
        xautrainLookUp[dst] = count
        # xautrain.append([0, int(autrain['dttl'].iloc[i]), int(autrain['dwin'].iloc[i])])
        xautrain.append([int(autrain['dttl'].iloc[i]), int(autrain['dwin'].iloc[i])])
        count += 1
    edge_index_autrain.append([xautrainLookUp[src], xautrainLookUp[dst]])
    edge_attr_autrain.append(list(autrain.iloc[i].drop(['sttl', 'dttl','swin','dwin','Label','stcpb','dtcpb']).values))



# Convert x to tensor
x_autrain_tensor = torch.tensor(xautrain, dtype=torch.float)

# Convert edge_index to tensor
edge_index_autrain_tensor = torch.tensor(edge_index_autrain, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_autrain_tensor = torch.tensor(edge_attr_autrain, dtype=torch.float)

print(x_autrain_tensor.shape, edge_index_autrain_tensor.shape, edge_attr_autrain_tensor.shape)

torch.Size([143645, 2]) torch.Size([2, 71928]) torch.Size([71928, 20])


## Author's Testing Dataset

In [15]:
autestnodes = set()
autestUnique = autest
for i in range(len(autest)):
    src = str(autest['sttl'].iloc[i])+':'+str(autest['swin'].iloc[i]) + str(autest['stcpb'].iloc[i])
    dst = str(autest['dttl'].iloc[i])+':'+str(autest['dwin'].iloc[i]) + str(autest['dtcpb'].iloc[i])
    autestnodes.add(src)
    autestnodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
autest_tuples = set()
for i in range(len(autest)):
    autest_tuples.add((autest['sttl'].iloc[i], autest['swin'].iloc[i], autest['dttl'].iloc[i], autest['dwin'].iloc[i], autest['stcpb'].iloc[i], autest['dtcpb'].iloc[i]))

In [16]:
xautestLookUp = {}
xautest = []
count = 0
edge_index_autest = []
edge_attr_autest = []

for i in range(len(autest)):
    src = str(autest['sttl'].iloc[i])+':'+str(autest['swin'].iloc[i]) + str(autest['stcpb'].iloc[i])
    dst = str(autest['dttl'].iloc[i])+':'+str(autest['dwin'].iloc[i]) + str(autest['dtcpb'].iloc[i])
    if src not in xautestLookUp:
        xautestLookUp[src] = count
        # xautest.append([0,int(autest['sttl'].iloc[i]), int(autest['swin'].iloc[i])])
        xautest.append([int(autest['sttl'].iloc[i]), int(autest['swin'].iloc[i])])
        count += 1
    if dst not in xautestLookUp:
        xautestLookUp[dst] = count
        # xautest.append([0, int(autest['dttl'].iloc[i]), int(autest['dwin'].iloc[i])])
        xautest.append([int(autest['dttl'].iloc[i]), int(autest['dwin'].iloc[i])])
        count += 1
    edge_index_autest.append([xautestLookUp[src], xautestLookUp[dst]])
    edge_attr_autest.append(list(autest.iloc[i].drop(['sttl', 'dttl','swin','dwin','Label','stcpb','dtcpb']).values))



# Convert x to tensor
x_autest_tensor = torch.tensor(xautest, dtype=torch.float)

# Convert edge_index to tensor
edge_index_autest_tensor = torch.tensor(edge_index_autest, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_autest_tensor = torch.tensor(edge_attr_autest, dtype=torch.float)

print(x_autest_tensor.shape, edge_index_autest_tensor.shape, edge_attr_autest_tensor.shape)

torch.Size([75920, 2]) torch.Size([2, 38025]) torch.Size([38025, 20])


# GAT Model

nn.Linear appplies linear transformation to the incoming data: y = x * transpose(A) + b 


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_channels, num_heads, num_classes):
        super(GATClassifier, self).__init__()
        self.conv1 = GATConv(num_node_features, hidden_channels, heads=num_heads, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, add_self_loops=False)
        self.edge_classifier = nn.Linear(hidden_channels * 2 * num_heads + num_edge_features, num_classes)

    def forward(self, x, edge_index, edge_attr):
        # Apply graph attentional layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Concatenate node features with edge features
        num_edges = edge_index.size(1)
        node_x = x[edge_index[0]]
        node_x_other = x[edge_index[1]]
        edge_attr_padded = torch.cat([edge_attr, torch.zeros(num_edges - edge_attr.size(0), edge_attr.size(1))], dim=0)
        edge_x = torch.cat([node_x, node_x_other, edge_attr_padded], dim=1)

        # Classify edges into classes
        edge_scores = self.edge_classifier(edge_x)
        return edge_scores

Training

In [18]:
# Example usage
num_nodes = len(nodes)
num_edges = len(train_tuples)
num_node_features = 2
num_edge_features = 20
hidden_channels = 16
num_heads = 32
num_classes = 26

# Instantiate GAT classifier
model = GATClassifier(num_node_features, num_edge_features, hidden_channels, num_heads, num_classes)

# Forward pass
edge_scores_train = model(x_tensor, edge_index_tensor, edge_attr_tensor)

Testing

In [19]:
# Example usage
num_nodes_test = len(testnodes)
num_edges_test = len(test_tuples)

# Forward pass
edge_scores_test = model(x_test_tensor, edge_index_test_tensor, edge_attr_test_tensor)

Author's training dataset

In [20]:
# Example usage
num_nodes_autrain = len(autrainnodes)
num_edges_autrain = len(autrain_tuples)

# Forward pass
edge_scores_autrain = model(x_autrain_tensor, edge_index_autrain_tensor, edge_attr_autrain_tensor)

Author's testing dataset

In [21]:
# Example usage
num_nodes_autest = len(autestnodes)
num_edges_autest = len(autest_tuples)

# Forward pass
edge_scores_autest = model(x_autest_tensor, edge_index_autest_tensor, edge_attr_autest_tensor)

# KNN

In [22]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
edge_scores_train_array = edge_scores_train.detach().numpy()
edge_scores_test_array = edge_scores_test.detach().numpy()
# Create a DataFrame from the NumPy array
columns = [str('Dim')+str(i) for i in range(edge_scores_train_array.shape[1])]
edge_scores_train_df = pd.DataFrame(edge_scores_train_array, columns=columns)
edge_scores_test_df = pd.DataFrame(edge_scores_test_array, columns=columns)
trainLabel = train['Label']
testLabel = test['Label']

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train = edge_scores_train_df
X_test = edge_scores_test_df
y_train = trainLabel
y_test = testLabel

# Initialize the KNN classifier
k = 10  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Calculate accuracy
report = classification_report(y_test, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     17201
           1       0.99      1.00      0.99     17229

    accuracy                           0.99     34430
   macro avg       0.99      0.99      0.99     34430
weighted avg       0.99      0.99      0.99     34430



## Attack Sequence Trace Back

In [26]:
testWithStimeLtime.iloc[0]['Stime']

-0.985842301525703

In [25]:
#Retrieve index positions of y_pred = 1
indices_of_ones = np.where(y_pred == 1)[0]

import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime

# Create a directed graph
traceBackG = nx.DiGraph()

for i in indices_of_ones:
    src = str(testWithStimeLtime.iloc[i]['srcip']) + str(testWithStimeLtime.iloc[i]['sport'])
    dst = str(testWithStimeLtime.iloc[i]['dstip']) + str(testWithStimeLtime.iloc[i]['dsport'])
    traceBackG.add_node(src)
    traceBackG.add_node(dst)
    stime = datetime.fromtimestamp(testWithStimeLtime.iloc[i]['Stime']).strftime('%Y-%m-%d %H:%M:%S')
    ltime = datetime.fromtimestamp(testWithStimeLtime.iloc[i]['Ltime']).strftime('%Y-%m-%d %H:%M:%S')
    traceBackG.add_edge(src, dst, label="Connection Start Time: " + str(stime) + ", Connection End Time: "+ str(ltime))


# Draw the graph
pos = nx.spring_layout(traceBackG)  # Define layout
nx.draw(traceBackG, pos, with_labels=True, node_size=1000, node_color='red', font_size=12, font_weight='bold')
node_labels = nx.get_node_attributes(traceBackG, 'label')
nx.draw_networkx_labels(traceBackG, pos, labels=node_labels, font_size=12, font_weight='bold', font_color='black', 
                        bbox=dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.3'))

# Show the plot
plt.show()

OSError: [Errno 22] Invalid argument

Author's test dataset

In [106]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
edge_scores_autest_array = edge_scores_autest.detach().numpy()
# Create a DataFrame from the NumPy array
#testLabeldf = pd.DataFrame(auEdgeLabel, columns=['Label'])
edge_scores_autest_df = pd.DataFrame(edge_scores_autest_array, columns=columns)
trainLabel = train['Label']
autestLabel = autest['Label']
auX_test = edge_scores_autest_df
auy_test = autestLabel 

auy_pred = knn_classifier.predict(edge_scores_autest_df)

# Calculate accuracy
report = classification_report(autestLabel, auy_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.65      0.92      0.77     24354
           1       0.49      0.13      0.21     13671

    accuracy                           0.64     38025
   macro avg       0.57      0.53      0.49     38025
weighted avg       0.59      0.64      0.56     38025



Purely using author's training and testing dataset

In [108]:
edge_scores_autrain_array = edge_scores_autrain.detach().numpy()
edge_scores_autest_array = edge_scores_autest.detach().numpy()

edge_scores_autrain_df = pd.DataFrame(edge_scores_autrain_array, columns=columns)
edge_scores_autest_df = pd.DataFrame(edge_scores_autest_array, columns=columns)


trainLabel = autrain['Label']
autestLabel = autest['Label']
auX_train = edge_scores_autrain_df
auX_test = edge_scores_autest_df

# Initialize the KNN classifier
k = 10  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(auX_train, trainLabel)

# Predictions on the test set
y_pred = knn_classifier.predict(auX_test)

# Calculate accuracy
report = classification_report(autestLabel, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.95      0.64      0.76     24354
           1       0.59      0.93      0.72     13671

    accuracy                           0.74     38025
   macro avg       0.77      0.79      0.74     38025
weighted avg       0.82      0.74      0.75     38025



# Lazy Predict Scores

## User prepared train-test

In [55]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm
import sklearn.discriminant_analysis
from lazypredict.Supervised import LazyClassifier


classifiers = [
    ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
    ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
    ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
    # ('ExtraTreeClassifier', sklearn.ensemble._forest.ExtraTreeClassifier),
    # ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
    # ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
    # ('NuSVC', sklearn.svm._classes.NuSVC),
    ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
    # ('SVC', sklearn.svm._classes.SVC),
    ('XGBClassifier', xgboost.sklearn.XGBClassifier),
    ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers= classifiers, predictions=True)

models,predictions = clf.fit(X_train, X_test, y_train, y_test)


'tuple' object has no attribute '__name__'
Invalid Classifier(s)


 17%|█▋        | 1/6 [00:05<00:29,  5.97s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.9422015683996514, 'Balanced Accuracy': 0.942209702776116, 'ROC AUC': 0.9422097027761159, 'F1 Score': 0.9421962032446103, 'Time taken': 5.965700626373291}


 33%|███▎      | 2/6 [00:15<00:31,  7.95s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.9885564914318908, 'Balanced Accuracy': 0.9885528935873277, 'ROC AUC': 0.9885528935873277, 'F1 Score': 0.988556227304697, 'Time taken': 9.343464851379395}


...

{'Model': 'RandomForestClassifier', 'Accuracy': 0.9901248910833576, 'Balanced Accuracy': 0.9901201132641712, 'ROC AUC': 0.9901201132641712, 'F1 Score': 0.9901245040339248, 'Time taken': 42.533103704452515}


 83%|████████▎ | 5/6 [00:59<00:12, 12.89s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.9892535579436538, 'Balanced Accuracy': 0.9892483071434326, 'ROC AUC': 0.9892483071434326, 'F1 Score': 0.9892530547831571, 'Time taken': 0.81636643409729}
[LightGBM] [Info] Number of positive: 39866, number of negative: 39990
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 79856, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499224 -> initscore=-0.003106
[LightGBM] [Info] Start training from score -0.003106


100%|██████████| 6/6 [01:00<00:00, 10.08s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9877432471681673, 'Balanced Accuracy': 0.9877366253952606, 'ROC AUC': 0.9877366253952606, 'F1 Score': 0.9877423554145359, 'Time taken': 0.7762844562530518}


In [56]:
from sklearn.metrics import roc_auc_score
for m in predictions.columns:
    print(m)
    print(classification_report(y_test, predictions[m]))
    auc_scores = roc_auc_score(y_test, predictions[m], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     17201
           1       0.95      0.93      0.94     17229

    accuracy                           0.94     34430
   macro avg       0.94      0.94      0.94     34430
weighted avg       0.94      0.94      0.94     34430

AUC Scores for each class: 0.9422097027761159
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

           0       0.99      0.98      0.99     17201
           1       0.98      0.99      0.99     17229

    accuracy                           0.99     34430
   macro avg       0.99      0.99      0.99     34430
weighted avg       0.99      0.99      0.99     34430

AUC Scores for each class: 0.9885528935873277
-------------------------------------------------

## Lazy Predict for author's train-test dataset

In [109]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm
import sklearn.discriminant_analysis
from lazypredict.Supervised import LazyClassifier

classifiers = [
    ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
    ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
    ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
    ('ExtraTreeClassifier', sklearn.ensemble._forest.ExtraTreeClassifier),
    ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
    ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
    ('NuSVC', sklearn.svm._classes.NuSVC),
    ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
    ('SVC', sklearn.svm._classes.SVC),
    ('XGBClassifier', xgboost.sklearn.XGBClassifier),
    ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers= classifiers, predictions=True)

aumodels, aupredictions = clf.fit(auX_train, auX_test, autrainLabel, autestLabel)

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  9%|▉         | 1/11 [00:32<05:22, 32.22s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.6816305062458908, 'Balanced Accuracy': 0.7450403617089016, 'ROC AUC': 0.7450403617089016, 'F1 Score': 0.6800791675542832, 'Time taken': 32.223750829696655}


 18%|█▊        | 2/11 [01:20<06:14, 41.60s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.7224457593688363, 'Balanced Accuracy': 0.7683526525551927, 'ROC AUC': 0.7683526525551927, 'F1 Score': 0.7257820911536753, 'Time taken': 48.15685486793518}


...

{'Model': 'SVC', 'Accuracy': 0.7021696252465484, 'Balanced Accuracy': 0.7668663059244154, 'ROC AUC': 0.7668663059244154, 'F1 Score': 0.7008627266358385, 'Time taken': 322.2111086845398}


 91%|█████████ | 10/11 [21:52<02:50, 170.23s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.7150821827744904, 'Balanced Accuracy': 0.7715562498354451, 'ROC AUC': 0.771556249835445, 'F1 Score': 0.7162727354250601, 'Time taken': 2.0797057151794434}
[LightGBM] [Info] Number of positive: 36961, number of negative: 34967
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015795 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6630
[LightGBM] [Info] Number of data points in the train set: 71928, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.513861 -> initscore=0.055459
[LightGBM] [Info] Start training from score 0.055459


100%|██████████| 11/11 [21:54<00:00, 119.47s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.7020118343195266, 'Balanced Accuracy': 0.7656361378683509, 'ROC AUC': 0.7656361378683509, 'F1 Score': 0.7010221200230649, 'Time taken': 1.764564037322998}


In [110]:
from sklearn.metrics import roc_auc_score
for model in aupredictions.columns:
    print(model)
    print(classification_report(autestLabel, aupredictions[model]))
    auc_scores = roc_auc_score(autestLabel, aupredictions[model], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.97      0.52      0.68     24354
           1       0.53      0.97      0.69     13671

    accuracy                           0.68     38025
   macro avg       0.75      0.75      0.68     38025
weighted avg       0.81      0.68      0.68     38025

AUC Scores for each class: 0.7450403617089016
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

           0       0.94      0.60      0.74     24354
           1       0.57      0.93      0.71     13671

    accuracy                           0.72     38025
   macro avg       0.76      0.77      0.72     38025
weighted avg       0.81      0.72      0.73     38025

AUC Scores for each class: 0.7683526525551927
-------------------------------------------------

In [111]:
aumodels

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
KNeighborsClassifier,0.75,0.79,0.79,0.75,14.69
XGBClassifier,0.72,0.77,0.77,0.72,2.08
BaggingClassifier,0.72,0.77,0.77,0.73,48.16
ExtraTreesClassifier,0.71,0.77,0.77,0.71,15.07
RandomForestClassifier,0.71,0.77,0.77,0.71,95.48
SVC,0.70,0.77,0.77,0.70,322.21
LGBMClassifier,0.70,0.77,0.77,0.70,1.76
ExtraTreeClassifier,0.72,0.75,0.75,0.72,0.29
AdaBoostClassifier,0.68,0.75,0.75,0.68,32.22


# GCN

In [112]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_channels, num_classes):
        super(GCNClassifier, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.edge_classifier = nn.Linear(hidden_channels * 2 + num_edge_features, num_classes)

    def forward(self, x, edge_index, edge_attr):
        # Apply graph convolutional layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Concatenate node features with edge features
        num_edges = edge_index.size(1)
        node_x = x[edge_index[0]]
        node_x_other = x[edge_index[1]]
        edge_attr_padded = torch.cat([edge_attr, torch.zeros(num_edges - edge_attr.size(0), edge_attr.size(1))], dim=0)
        edge_x = torch.cat([node_x, node_x_other, edge_attr_padded], dim=1)

        # Classify edges into classes
        edge_scores = self.edge_classifier(edge_x)
        return edge_scores

## Training using user prepared dataset

In [117]:
num_nodes = len(nodes)
num_edges = len(train_tuples)
num_node_features = 2
num_edge_features = 20
hidden_channels = 8
num_classes = 26

GCNmodel1 = GCNClassifier(num_node_features, num_edge_features, hidden_channels, num_classes)


GCNedge_scores_train = GCNmodel1(x_tensor, edge_index_tensor, edge_attr_tensor)

## Testing with user prepared dataset

In [118]:
num_nodes_test = len(testnodes)
num_edges_test = len(test_tuples)

# Forward pass
GCNedge_scores_test = GCNmodel1(x_test_tensor, edge_index_test_tensor, edge_attr_test_tensor)

## Training using Author's Train set

In [119]:
num_nodes_autrain = len(autrainnodes)
num_edges_autrain = len(autrain_tuples)
num_node_features = 2
num_edge_features = 20
hidden_channels = 8
num_classes = 26

GCNmodel2 = GCNClassifier(num_node_features, num_edge_features, hidden_channels, num_classes)
GCNedge_scores_autrain = GCNmodel2(x_autrain_tensor, edge_index_autrain_tensor, edge_attr_autrain_tensor)

## Testing using Author's Test set

In [120]:
# Example usage
num_nodes_autest = len(autestnodes)
num_edges_autest = len(autest_tuples)


# Forward pass
GCNedge_scores_autest = GCNmodel2(x_autest_tensor, edge_index_autest_tensor, edge_attr_autest_tensor)

## KNN

In [121]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
GCNedge_scores_train_array = GCNedge_scores_train.detach().numpy()
GCNedge_scores_test_array = GCNedge_scores_test.detach().numpy()
# Create a DataFrame from the NumPy array
columns = [str('Dim')+str(i) for i in range(GCNedge_scores_train_array.shape[1])]
GCNedge_scores_train_df = pd.DataFrame(GCNedge_scores_train_array, columns=columns)
GCNedge_scores_test_df = pd.DataFrame(GCNedge_scores_test_array, columns=columns)
trainLabel = train['Label']
testLabel = test['Label']

In [123]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

GCNX_train = GCNedge_scores_train_df
GCNX_test = GCNedge_scores_test_df
y_train = trainLabel
y_test = testLabel

# Initialize the KNN classifier
k = 10  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(GCNX_train, y_train)

# Predictions on the test set
y_pred = knn_classifier.predict(GCNX_test)

# Calculate accuracy
report = classification_report(y_test, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     17201
           1       0.99      1.00      0.99     17229

    accuracy                           0.99     34430
   macro avg       0.99      0.99      0.99     34430
weighted avg       0.99      0.99      0.99     34430



In [124]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
GCNedge_scores_autest_array = GCNedge_scores_autest.detach().numpy()
# Create a DataFrame from the NumPy array
#testLabeldf = pd.DataFrame(auEdgeLabel, columns=['Label'])
GCNedge_scores_autest_df = pd.DataFrame(GCNedge_scores_autest_array, columns=columns)
trainLabel = train['Label']
autestLabel = autest['Label']
auX_test = GCNedge_scores_autest_df
auy_test = autestLabel 

auy_pred = knn_classifier.predict(GCNedge_scores_autest_df)

# Calculate accuracy
report = classification_report(autestLabel, auy_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.65      0.96      0.78     24354
           1       0.58      0.09      0.15     13671

    accuracy                           0.65     38025
   macro avg       0.61      0.53      0.46     38025
weighted avg       0.63      0.65      0.55     38025



In [125]:
GCNedge_scores_autrain_array = GCNedge_scores_autrain.detach().numpy()
GCNedge_scores_autest_array = GCNedge_scores_autest.detach().numpy()

GCNedge_scores_autrain_df = pd.DataFrame(GCNedge_scores_autrain_array, columns=columns)
GCNedge_scores_autest_df = pd.DataFrame(GCNedge_scores_autest_array, columns=columns)


trainLabel = autrain['Label']
autestLabel = autest['Label']
auX_train = GCNedge_scores_autrain_df
auX_test = GCNedge_scores_autest_df

# Initialize the KNN classifier
k = 10  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(auX_train, trainLabel)

# Predictions on the test set
y_pred = knn_classifier.predict(auX_test)

# Calculate accuracy
report = classification_report(autestLabel, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.67      0.79     24354
           1       0.61      0.92      0.74     13671

    accuracy                           0.76     38025
   macro avg       0.78      0.80      0.76     38025
weighted avg       0.82      0.76      0.77     38025



# GraphSAGE

In [129]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
class GraphSAGEClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_channels, num_classes):
        super(GraphSAGEClassifier, self).__init__()
        self.conv1 = SAGEConv(num_node_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.edge_classifier = nn.Linear(hidden_channels * 2 + num_edge_features, num_classes)

    def forward(self, x, edge_index, edge_attr):
        # Apply GraphSAGE convolutional layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Concatenate node features with edge features
        num_edges = edge_index.size(1)
        node_x = x[edge_index[0]]
        node_x_other = x[edge_index[1]]
        edge_attr_padded = torch.cat([edge_attr, torch.zeros(num_edges - edge_attr.size(0), edge_attr.size(1))], dim=0)
        edge_x = torch.cat([node_x, node_x_other, edge_attr_padded], dim=1)

        # Classify edges into binary class using BCELogit
        edge_scores = self.edge_classifier(edge_x)
        return edge_scores

In [130]:
num_nodes = len(nodes)
num_edges = len(train_tuples)
num_node_features = 2
num_edge_features = 20
hidden_channels = 8
num_classes = 26

GraphSAGEmodel1 = GraphSAGEClassifier(num_node_features, num_edge_features, hidden_channels, num_classes)


GraphSAGEedge_scores_train = GraphSAGEmodel1(x_tensor, edge_index_tensor, edge_attr_tensor)

In [131]:
num_nodes_test = len(testnodes)
num_edges_test = len(test_tuples)

# Forward pass
GraphSAGEedge_scores_test = GraphSAGEmodel1(x_test_tensor, edge_index_test_tensor, edge_attr_test_tensor)

In [132]:
num_nodes_autrain = len(autrainnodes)
num_edges_autrain = len(autrain_tuples)
num_node_features = 2
num_edge_features = 20
hidden_channels = 8
num_classes = 26

GraphSAGEmodel2 = GraphSAGEClassifier(num_node_features, num_edge_features, hidden_channels, num_classes)
GraphSAGEedge_scores_autrain = GraphSAGEmodel2(x_autrain_tensor, edge_index_autrain_tensor, edge_attr_autrain_tensor)

In [133]:
# Example usage
num_nodes_autest = len(autestnodes)
num_edges_autest = len(autest_tuples)


# Forward pass
GraphSAGEedge_scores_autest = GraphSAGEmodel2(x_autest_tensor, edge_index_autest_tensor, edge_attr_autest_tensor)

## KNN

In [134]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
GraphSAGEedge_scores_train_array = GraphSAGEedge_scores_train.detach().numpy()
GraphSAGEedge_scores_test_array = GraphSAGEedge_scores_test.detach().numpy()
# Create a DataFrame from the NumPy array
columns = [str('Dim')+str(i) for i in range(GraphSAGEedge_scores_train_array.shape[1])]
GraphSAGEedge_scores_train_df = pd.DataFrame(GraphSAGEedge_scores_train_array, columns=columns)
GraphSAGEedge_scores_test_df = pd.DataFrame(GraphSAGEedge_scores_test_array, columns=columns)
trainLabel = train['Label']
testLabel = test['Label']

In [135]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

GraphSAGEX_train = GraphSAGEedge_scores_train_df
GraphSAGEX_test = GraphSAGEedge_scores_test_df
y_train = trainLabel
y_test = testLabel

# Initialize the KNN classifier
k = 10  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(GraphSAGEX_train, y_train)

# Predictions on the test set
y_pred = knn_classifier.predict(GraphSAGEX_test)

# Calculate accuracy
report = classification_report(y_test, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     17201
           1       0.99      1.00      0.99     17229

    accuracy                           0.99     34430
   macro avg       0.99      0.99      0.99     34430
weighted avg       0.99      0.99      0.99     34430



In [136]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
GraphSAGEedge_scores_autest_array = GraphSAGEedge_scores_autest.detach().numpy()
# Create a DataFrame from the NumPy array
#testLabeldf = pd.DataFrame(auEdgeLabel, columns=['Label'])
GraphSAGEedge_scores_autest_df = pd.DataFrame(GraphSAGEedge_scores_autest_array, columns=columns)
trainLabel = train['Label']
autestLabel = autest['Label']
auX_test = GraphSAGEedge_scores_autest_df
auy_test = autestLabel 

auy_pred = knn_classifier.predict(GraphSAGEedge_scores_autest_df)

# Calculate accuracy
report = classification_report(autestLabel, auy_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.65      0.99      0.78     24354
           1       0.73      0.05      0.09     13671

    accuracy                           0.65     38025
   macro avg       0.69      0.52      0.44     38025
weighted avg       0.68      0.65      0.53     38025



In [137]:
GraphSAGEedge_scores_autrain_array = GraphSAGEedge_scores_autrain.detach().numpy()
GraphSAGEedge_scores_autest_array = GraphSAGEedge_scores_autest.detach().numpy()

GraphSAGEedge_scores_autrain_df = pd.DataFrame(GraphSAGEedge_scores_autrain_array, columns=columns)
GraphSAGEedge_scores_autest_df = pd.DataFrame(GraphSAGEedge_scores_autest_array, columns=columns)


trainLabel = autrain['Label']
autestLabel = autest['Label']
auX_train = GraphSAGEedge_scores_autrain_df
auX_test = GraphSAGEedge_scores_autest_df

# Initialize the KNN classifier
k = 10  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(auX_train, trainLabel)

# Predictions on the test set
y_pred = knn_classifier.predict(auX_test)

# Calculate accuracy
report = classification_report(autestLabel, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.67      0.78     24354
           1       0.61      0.92      0.73     13671

    accuracy                           0.76     38025
   macro avg       0.77      0.80      0.76     38025
weighted avg       0.82      0.76      0.76     38025

